<a href="https://colab.research.google.com/github/athospugliesedev/openai_codes/blob/main/openai_completion_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install openai

In [11]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [12]:
from openai import OpenAI, AssistantEventHandler
from typing_extensions import override

# Inicializa o cliente OpenAI
client = OpenAI()

# Solicita ao usuário que insira o código em R
r_code = input("Digite o código em R: ")

# Define o conteúdo da mensagem do usuário com base no código em R inserido
user_message_content = f"""Convert the R code in the following markdown code block into Python. Pay particular attention to style, and try to use imports correctly. Render all output code between "```python" markdown code blocks:

```R

{r_code}

```"""

# Cria a mensagem do usuário
user_message = {"role": "user", "content": user_message_content}

# Adiciona a mensagem do usuário à lista de mensagens
messages = [
    {"role": "system", "content": "You are a highly experienced developer working on convert Data Science project, an project that helps translate the default code in R to Python. It has two versions: R, Python and PySpark. You are well-versed in both languages, and help keep both versions in sync."},
    user_message
]

# Cria a conclusão com as mensagens atualizadas
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

# Imprime a resposta gerada pelo ChatGPT
print(completion.choices[0].message.content)

python_code = completion.choices[0].message.content

# Cria um assistente com base nas instruções fornecidas
my_assistant = client.beta.assistants.create(
    instructions="You are a personal python tutor. When asked a question, you change the default values and run the code to answer the question.",
    name="Python Tutor",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
)

# Inicializa o ID do assistente criado
assistant_id = my_assistant.id

# Define o ID do thread
thread = client.beta.threads.create()

# Cria uma mensagem com o conteúdo gerado pelo modelo anterior
message_content = completion.choices[0].message.content
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message_content
)

# Define um manipulador de eventos personalizado para o assistente
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Cria o manipulador de eventos
event_handler = EventHandler()

# Inicia o stream de execução do assistente
instructions = f"Change the default values from the dataframe, and do python tests and print the test and the result. {python_code}"
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant_id,
    instructions=instructions,
    event_handler=event_handler,
) as stream:
    stream.until_done()


Digite o código em R: # Instale e carregue o pacote 'httr' para fazer solicitações HTTP install.packages("httr") library(httr)  # Dados de exemplo em R dados <- data.frame(   x = c(1, 2, 3, 4, 5),   y = c(10, 20, 30, 40, 50) )  # Converta os dados para JSON dados_json <- toJSON(dados)  # URL da API do AIMMS onde você deseja enviar os dados url <- "http://exemplo.com/api/receber_dados"  # Faça uma solicitação POST para enviar os dados para o AIMMS resposta <- POST(url, body = dados_json, encode = "json")  # Verifique o status da resposta if (http_status(resposta)$status == 200) {   print("Dados enviados com sucesso para o AIMMS.") } else {   print("Erro ao enviar dados para o AIMMS.") }
```python
import requests
import json

# Dados de exemplo em Python
dados = {
    'x': [1, 2, 3, 4, 5],
    'y': [10, 20, 30, 40, 50]
}

# Converta os dados para JSON
dados_json = json.dumps(dados)

# URL da API do AIMMS onde você deseja enviar os dados
url = "http://exemplo.com/api/receber_dados"

# Faç